In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
#from torchvision import Datasets
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
import spacy

C:\Users\maddo\anaconda3\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
spacy_en = spacy.load('en_core_web_sm')
spacy_fr = spacy.load('fr_core_news_sm')

In [3]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

In [4]:
data=pd.read_csv('engfrench.csv')

In [5]:
print(len(data))

175621


In [6]:
data.iloc[170000]

English words/sentences    The guards found a hacksaw blade in the prison...
French words/sentences     Les gardiens trouvèrent une lame de scie à mét...
Name: 170000, dtype: object

In [7]:
data[10000:10005]

,English words/sentences,French words/sentences
10000,Be very careful.,Sois très prudente !
10001,Bees make honey.,Les abeilles font du miel.
10002,Behave yourself.,Comporte-toi bien.
10003,Bite the bullet.,Serre les dents.
10004,Bite the bullet.,Serrez les dents.


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [8]:
PAD_TOKEN_ID=0
SOS_TOKEN_ID = 1
EOS_TOKEN_ID = 2

In [9]:
txten=data.iloc[0,0]
txtfr=data.iloc[0,1]


In [10]:
txten

'Hi.'

In [11]:
txtfr

'Salut!'

In [12]:
en_check=torch.load('engEmbeddings.pt')
fr_check=torch.load('frEmbeddings.pt')
en_weights=en_check['weights']
en_vocab=en_check['vocab_dict']
fr_weights=fr_check['weights']
fr_vocab=fr_check['vocab_dict']

C:\Users\maddo\AppData\Local\Temp\ipykernel_17304\1505539583.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  en_check=torch.load('engEmbeddings.pt')
C:\Users\maddo\AppDa

In [13]:
class TranslationDataset(Dataset):
    def __init__(self, dataframe, en_vocab, fr_vocab):
        self.df = dataframe
        self.en_vocab = en_vocab
        self.fr_vocab = fr_vocab
        
        # Define special token IDs
        # IMPORTANT: Ensure these keys exist in your vocab dictionaries!
        self.unk_idx_en = en_vocab.get('<unk>', 0) # Default to 0 if not found
        self.eos_idx_en = en_vocab.get('<eos>', 2)
        
        self.unk_idx_fr = fr_vocab.get('<unk>', 0)
        self.sos_idx_fr = fr_vocab.get('<sos>', 1)
        self.eos_idx_fr = fr_vocab.get('<eos>', 2)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # 1. Get Raw Text
        src_text = str(self.df.iloc[index, 0])
        trg_text = str(self.df.iloc[index, 1])

        # 2. Tokenize (using your Spacy functions)
        # We use .lower() because FastText matches better with lowercase
        src_tokens = tokenize_en(src_text.lower())
        trg_tokens = tokenize_fr(trg_text.lower())

        # 3. Numericalize Source (English)
        # Use .get() to map unknown words to <unk> index
        src_indices = [self.en_vocab.get(token, self.unk_idx_en) for token in src_tokens]
        
        # 4. Numericalize Target (French)
        trg_indices = [self.fr_vocab.get(token, self.unk_idx_fr) for token in trg_tokens]

        # 5. Add Special Tokens
        # Source usually needs <eos> at the end so the LSTM knows the sentence stopped
        src_indices.append(self.eos_idx_en)
        
        # Target needs <sos> to start generation and <eos> to end it
        trg_indices = [self.sos_idx_fr] + trg_indices + [self.eos_idx_fr]

        return torch.tensor(src_indices, dtype=torch.long), torch.tensor(trg_indices, dtype=torch.long)

In [14]:
datanewdataset=TranslationDataset(data,en_vocab,fr_vocab)

In [15]:
def custom_collate_fn(batch):
    # 'batch' is a list of tuples: [(input_tensor_1, target_tensor_1), (input_tensor_2, target_tensor_2), ...]

    # 1. Separate inputs and targets
    inputs = [torch.tensor(item[0]) for item in batch]
    targets = [torch.tensor(item[1]) for item in batch]

    # 2. Pad the inputs and targets to the length of the longest sequence in the batch
    # batch_first=True makes the output shape (BatchSize, SequenceLength)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=PAD_TOKEN_ID)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=PAD_TOKEN_ID)
    
    return inputs_padded, targets_padded

In [16]:
haha=DataLoader(datanewdataset,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)

In [17]:
xa=torch.randn(10,100)

In [18]:
ha=nn.LSTM(100,200)

In [19]:
ya=ha(xa)

In [20]:
ya[0].shape

torch.Size([10, 200])

In [21]:
ya[1][0].shape

torch.Size([1, 200])

In [22]:
ya[1][1].shape

torch.Size([1, 200])

In [23]:
embed_dim=300
#since using pre trained Fast TExt aligned vectors

In [24]:
VOCAB_SIZE_ENC=len(en_vocab)

In [25]:
VOCAB_SIZE_FR=len(fr_vocab)

In [26]:
class enc(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding.from_pretrained(en_weights, freeze=True)
        self.lstm=nn.LSTM(input_size=embed_dim,hidden_size=1024,num_layers=3,batch_first=True,bidirectional=False)
        self.rel=nn.ReLU()
    def forward(self,x):
        embx=self.emb(x)
        out,(hid,cell)=self.lstm(embx)
        return hid,cell
        

        
        

In [27]:
class dec(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding.from_pretrained(fr_weights, freeze=True)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=1024, num_layers=3, batch_first=True, bidirectional=False)
        self.fc_out = nn.Linear(1024, VOCAB_SIZE_FR) # Map hidden state to Vocab Size
        
    def forward(self, x, hidden, cell):
        # x shape: [batch_size] -> unsqueeze to [batch_size, 1]
        x = x.unsqueeze(1)
        embx = self.emb(x)
        
        output, (hidden, cell) = self.lstm(embx, (hidden, cell))
        
        # prediction shape: [batch_size, VOCAB_SIZE]
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        # source: [batch, src_len]
        # target: [batch, trg_len]
        
        batch_size = source.shape[0]
        target_len = target.shape[1]
        vocab_size = len(fr_vocab)
        
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        
        # 1. Encode
        hidden, cell = self.encoder(source)
        
        # 2. First input to decoder (Start Token)
        # We use the SOS_TOKEN_ID defined at the top
        input_token = torch.full((batch_size,), SOS_TOKEN_ID, dtype=torch.long).to(self.device)
        
        # 3. Decode
        for t in range(target_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t, :] = output
            
            # Teacher forcing
            top1 = output.argmax(1)
            use_teacher = random.random() < teacher_forcing_ratio
            
            # If using teacher forcing, next input is actual target token
            # Else, next input is predicted token
            if t < target_len - 1:
                input_token = target[:, t] if use_teacher else top1
            
        return outputs

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

encoder = enc()
decoder = dec()
model = Seq2Seq(encoder, decoder, device).to(device)

# Optimization
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN_ID)

Using device: cuda


In [30]:
def train_one_epoch(model, iterator, optimizer, criterion,epoch, clip=1):
    model.train()
    epoch_loss = 0
    for kek in range(epoch):
        for i, (src, trg) in enumerate(iterator):
            src = src.to(device)
            trg = trg.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            output = model(src, trg)
            
            # Reshape for Loss
            # output: [batch, len, vocab] -> [batch*len, vocab]
            # trg: [batch, len] -> [batch*len]
            output_dim = output.shape[-1]
            output = output.view(-1, output_dim)
            trg = trg.view(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            
            # Clip gradients to prevent explosion
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            
            epoch_loss += loss.item()
            
            if i % 500 == 0:
                print(f"Epoch number {kek} batch {i}, Loss: {loss.item()}")
                
        print( epoch_loss / len(iterator))

In [31]:
def predict_translation(model, sentence, max_len=20):
    model.eval()
    
    # ---------------------------------------------
    # 1. Preprocess the Input (English)
    # ---------------------------------------------
    # Use the same tokenization function as training
    tokens = tokenize_en(sentence.lower())
    
    # Manually convert tokens to IDs using en_vocab
    # Handle unknown words with .get(token, unk_index)
    unk_idx = en_vocab.get('<unk>', 3) 
    eos_idx = en_vocab.get('<eos>', 2)
    
    ids = [en_vocab.get(token, unk_idx) for token in tokens]
    
    # IMPORTANT: Add <eos> if your model was trained with it
    ids.append(eos_idx)
    
    src_tensor = torch.tensor(ids, dtype=torch.long).unsqueeze(0).to(device) # [1, seq_len]

    # ---------------------------------------------
    # 2. Encode
    # ---------------------------------------------
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # ---------------------------------------------
    # 3. Decode Loop
    # ---------------------------------------------
    # Get special tokens for French
    sos_idx = fr_vocab['<sos>']
    eos_idx = fr_vocab['<eos>']
    
    inputs = [sos_idx]

    for _ in range(max_len):
        input_tensor = torch.tensor([inputs[-1]], dtype=torch.long).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(input_tensor, hidden, cell)
        
        pred_token = output.argmax(1).item()
        
        # Stop if EOS is predicted
        if pred_token == eos_idx:
            break
        
        inputs.append(pred_token)

    # ---------------------------------------------
    # 4. Convert IDs back to Words (French)
    # ---------------------------------------------
    # We need to invert the fr_vocab dictionary (ID -> Word)
    # ideally, do this once globally, but here is fine for inference
    idx_to_word = {v: k for k, v in fr_vocab.items()}
    
    # Skip the first token (which is SOS)
    predicted_tokens = [idx_to_word.get(idx, '<unk>') for idx in inputs[1:]]
    
    return " ".join(predicted_tokens)

In [32]:
# Example run (Uncomment to train)
print("Starting Training...")
loss = train_one_epoch(model, haha, optimizer, criterion,epoch=10)
print(f"Epoch Loss: {loss}")

Starting Training...


C:\Users\maddo\AppData\Local\Temp\ipykernel_17304\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_17304\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]


Epoch number 0 batch 0, Loss: 10.111145973205566
Epoch number 0 batch 500, Loss: 4.8358283042907715
Epoch number 0 batch 1000, Loss: 4.240336894989014
Epoch number 0 batch 1500, Loss: 4.01328706741333
Epoch number 0 batch 2000, Loss: 3.7634356021881104
Epoch number 0 batch 2500, Loss: 4.302215576171875
Epoch number 0 batch 3000, Loss: 3.6970086097717285
Epoch number 0 batch 3500, Loss: 3.7250916957855225
Epoch number 0 batch 4000, Loss: 3.2472798824310303
Epoch number 0 batch 4500, Loss: 3.327038049697876
Epoch number 0 batch 5000, Loss: 3.6872339248657227
3.8949108085990236
Epoch number 1 batch 0, Loss: 3.3789267539978027
Epoch number 1 batch 500, Loss: 3.486088752746582
Epoch number 1 batch 1000, Loss: 3.4611101150512695
Epoch number 1 batch 1500, Loss: 3.2933602333068848
Epoch number 1 batch 2000, Loss: 2.9352400302886963
Epoch number 1 batch 2500, Loss: 3.0420103073120117
Epoch number 1 batch 3000, Loss: 2.840574264526367
Epoch number 1 batch 3500, Loss: 2.8554234504699707
Epoch nu

In [41]:
translation = predict_translation(model, "I am doing great , WHat about you?")
print(f"Translation: {translation}")

Translation: <sos> je suis parfaite . c' est tout toi   ?


In [42]:
translation = predict_translation(model, "Today's a good day.")
print(f"Translation: {translation}")

Translation: <sos> c' est une bonne journée .


In [50]:
translation = predict_translation(model, "How was prison?")
print(f"Translation: {translation}")

Translation: <sos> comment s' est passé   ?


In [44]:
translation = predict_translation(model, "Bite the bullet.")
print(f"Translation: {translation}")

Translation: <sos> coupe la dents .


In [47]:
translation = predict_translation(model, "what's the time?")
print(f"Translation: {translation}")

Translation: <sos> quelle est la heure ?


In [48]:
translation = predict_translation(model, "what's the time")
print(f"Translation: {translation}")

Translation: <sos> quelle est le   ?


In [49]:
translation = predict_translation(model, "whats the time?")
print(f"Translation: {translation}")

Translation: <sos> quel musée est -t -elle   ?


In [51]:
translation = predict_translation(model, "behave yourself")
print(f"Translation: {translation}")

Translation: <sos> comporte - toi   !


In [53]:
translation = predict_translation(model, "please stop!")
print(f"Translation: {translation}")

Translation: <sos> s' il vous plaît , arrêtez .


In [54]:
translation = predict_translation(model, "please stop")
print(f"Translation: {translation}")

Translation: <sos> je vous prie !
